In [151]:
import pandas as pd
import numpy as np
import datetime
import time
import json
from geojson import Feature, FeatureCollection, Point

# Predictions Data


In [152]:
original_predictions = pd.read_csv('../notebooks/final_result_2022_07_05-2022_07_30.csv' )
original_predictions.head()

,Unnamed: 0,day0,day_confidence0,day1,day_confidence1,day2,day_confidence2,day3,day_confidence3,day4,...,day22,day_confidence22,day23,day_confidence23,day24,day_confidence24,day25,day_confidence25,latitude,longitude
0,0,0,0.0,0,0.0,0,0.000000,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.727215
1,1,0,0.0,0,0.0,1,0.342601,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.714526
2,2,0,0.0,0,0.0,1,0.342601,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.676457
3,3,0,0.0,0,0.0,1,0.342601,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.625699
4,4,0,0.0,0,0.0,1,0.342601,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.613010


In [153]:
original_predictions = original_predictions.iloc[:, -12:]
original_predictions

,day21,day_confidence21,day22,day_confidence22,day23,day_confidence23,day24,day_confidence24,day25,day_confidence25,latitude,longitude
0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.727215
1,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.714526
2,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.676457
3,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.625699
4,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.613010
...,...,...,...,...,...,...,...,...,...,...,...,...
129799,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.555160,-122.245014
129800,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.542471,-122.359220
129801,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.542471,-122.333841
129802,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.542471,-122.321151


In [154]:
column_names = []

for i in range(5):
    column_names.extend([f'day{i}', f'day_confidence{i}'])
column_names.extend(['latitude', 'longitude'])

original_predictions.columns = pd.Series(column_names)
original_predictions

,day0,day_confidence0,day1,day_confidence1,day2,day_confidence2,day3,day_confidence3,day4,day_confidence4,latitude,longitude
0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.727215
1,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.714526
2,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.676457
3,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.625699
4,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.613010
...,...,...,...,...,...,...,...,...,...,...,...,...
129799,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.555160,-122.245014
129800,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.542471,-122.359220
129801,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.542471,-122.333841
129802,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.542471,-122.321151


In [155]:
cur_date = datetime.date(2022, 7, 26)
delta = datetime.timedelta(days = 1)


In [156]:
days = []
nlargest = 2000

for i in range(5):
    cur_day = (original_predictions[[f"day{i}", f"day_confidence{i}", "latitude", "longitude"]]).dropna()
    cur_date_col = pd.DataFrame([cur_date]*len(cur_day.index), columns=["date"])
    cur_day = cur_date_col.join(cur_day)
    cur_date += delta
    cur_day = cur_day.nlargest(nlargest, f'day_confidence{i}')
    cur_day.columns = pd.Series(['date', 'dayResult', 'dayConfidence', 'latitude', 'longitude'])
    cur_day = cur_day[cur_day['dayResult'] != 0]
    days.append(cur_day)



In [157]:
all_predictions = pd.concat(days)
all_predictions = all_predictions[all_predictions["dayResult"] == 1]
all_predictions.insert(0, "SNo", range(0, len(all_predictions)))
all_predictions["dateUnixTimestamp"] = all_predictions.apply( lambda r: int(time.mktime(r["date"].timetuple())), axis=1 )
all_predictions.drop(columns=['date', 'SNo', 'dayResult'], inplace=True)
all_predictions.reset_index(drop=True, inplace=True)
all_predictions

,dayConfidence,latitude,longitude,dateUnixTimestamp
0,0.728846,47.699686,-123.171347,1658793600
1,0.728846,47.686996,-123.234795,1658793600
2,0.728846,47.686996,-123.222105,1658793600
3,0.728846,47.686996,-123.184037,1658793600
4,0.728846,47.686996,-123.171347,1658793600
...,...,...,...,...
4995,0.477894,46.951005,-122.206946,1659139200
4996,0.477894,46.951005,-122.194256,1659139200
4997,0.477894,46.951005,-122.181567,1659139200
4998,0.477894,46.951005,-122.168877,1659139200


In [158]:
all_predictions['date'] = all_predictions['dateUnixTimestamp']
all_predictions.drop(columns='dateUnixTimestamp', inplace=True)
all_predictions

,dayConfidence,latitude,longitude,date
0,0.728846,47.699686,-123.171347,1658793600
1,0.728846,47.686996,-123.234795,1658793600
2,0.728846,47.686996,-123.222105,1658793600
3,0.728846,47.686996,-123.184037,1658793600
4,0.728846,47.686996,-123.171347,1658793600
...,...,...,...,...
4995,0.477894,46.951005,-122.206946,1659139200
4996,0.477894,46.951005,-122.194256,1659139200
4997,0.477894,46.951005,-122.181567,1659139200
4998,0.477894,46.951005,-122.168877,1659139200


In [159]:
all_predictions.set_index(['date','latitude', 'longitude'], inplace=True)
all_predictions

dayConfidence
date       latitude  longitude                 
1658793600 47.699686 -123.171347       0.728846
           47.686996 -123.234795       0.728846
                     -123.222105       0.728846
                     -123.184037       0.728846
                     -123.171347       0.728846
...                                         ...
1659139200 46.951005 -122.206946       0.477894
                     -122.194256       0.477894
                     -122.181567       0.477894
                     -122.168877       0.477894
                     -122.156188       0.477894

[5000 rows x 1 columns]

# Historical Data


In [160]:
historical_data = pd.read_csv('../notebooks/processed_wa_fire_2022_July.csv' )
historical_data

,Unnamed: 0,InitialLatitude,InitialLongitude,FireDiscoveryDateTime,date,nearest_lat,nearest_lon,nearest_index
0,0,48.569070,-118.10940,2022/07/05 01:47:59+00,2022/07/05,48.575261,-118.108238,15124
1,1,47.803720,-117.10540,2022/07/05 02:19:56+00,2022/07/05,47.801202,-117.105767,46069
2,2,46.182840,-119.87170,2022/07/05 02:41:00+00,2022/07/05,46.176946,-119.872078,116796
3,3,48.193800,-120.50170,2022/07/05 15:09:54+00,2022/07/05,48.194576,-120.506553,28261
4,4,48.281330,-118.33200,2022/07/05 06:41:00+00,2022/07/05,48.283403,-118.336649,25132
...,...,...,...,...,...,...,...,...
198,198,46.958210,-122.19040,2022/07/30 16:59:06+00,2022/07/30,46.963695,-122.194256,82600
199,199,48.997167,-122.71610,2022/07/13 17:45:47+00,2022/07/13,48.994014,-122.714526,157
200,200,48.996960,-122.71610,2022/07/13 22:46:11+00,2022/07/13,48.994014,-122.714526,157
201,201,48.763420,-122.62890,2022/07/15 15:19:00+00,2022/07/15,48.765603,-122.625699,8295


In [161]:
historical_data["dateUnixTimestamp"] = historical_data.apply( lambda r: int(time.mktime(datetime.datetime.strptime(r['date'], "%Y/%m/%d").timetuple())) , axis=1 )
# historical_data = historical_data['InitialLatitude', 'InitialLongitude', 'date', 'dateUnixTimestamp']
historical_data['dayConfidence'] = '-1'
historical_data.sort_values(by='dateUnixTimestamp', inplace=True)
# historical_data = historical_data[['InitialLatitude', 'InitialLongitude','dayConfidence', 'dateUnixTimestamp', 'date']]
historical_data = historical_data[['nearest_lat', 'nearest_lon','dayConfidence', 'dateUnixTimestamp', 'date']]
historical_data.columns = pd.Series(['latitude', 'longitude', 'dayConfidence', 'dateUnixTimestamp', 'date'])
historical_data

,latitude,longitude,dayConfidence,dateUnixTimestamp,date
0,48.575261,-118.108238,-1,1656979200,2022/07/05
1,47.801202,-117.105767,-1,1656979200,2022/07/05
2,46.176946,-119.872078,-1,1656979200,2022/07/05
3,48.194576,-120.506553,-1,1656979200,2022/07/05
4,48.283403,-118.336649,-1,1656979200,2022/07/05
...,...,...,...,...,...
184,48.714845,-118.133617,-1,1659139200,2022/07/30
183,47.509343,-121.064891,-1,1659139200,2022/07/30
180,47.179416,-120.595379,-1,1659139200,2022/07/30
198,46.963695,-122.194256,-1,1659139200,2022/07/30


In [162]:
# drop dates above 25th

all_historical = historical_data[historical_data['date'] <= '2022/07/25']
all_historical = all_historical.drop(columns='date')
all_historical = all_historical.rename({'dateUnixTimestamp':'date'}, axis=1)
all_historical = all_historical.set_index(['date', 'latitude', 'longitude'])
# all_historical = all_historical.set_index(['idx'])
all_historical

dayConfidence
date       latitude  longitude                
1656979200 48.575261 -118.108238            -1
           47.801202 -117.105767            -1
           46.176946 -119.872078            -1
           48.194576 -120.506553            -1
           48.283403 -118.336649            -1
...                                        ...
1658707200 48.321471 -117.879827            -1
           47.648928 -117.448384            -1
           47.801202 -121.293302            -1
           48.042302 -118.894987            -1
           47.179416 -123.145968            -1

[161 rows x 1 columns]

In [163]:
print(all_historical.index.get_level_values(0).nunique())
print(all_historical.index.get_level_values(0).unique())
print(all_historical.index.get_level_values(0).unique())
print([datetime.datetime.fromtimestamp(int(i)) for i in all_historical.index.get_level_values(0).unique()])

21
Int64Index([1656979200, 1657065600, 1657152000, 1657238400, 1657324800,
            1657411200, 1657497600, 1657584000, 1657670400, 1657756800,
            1657843200, 1657929600, 1658016000, 1658102400, 1658188800,
            1658275200, 1658361600, 1658448000, 1658534400, 1658620800,
            1658707200],
           dtype='int64', name='date')
Int64Index([1656979200, 1657065600, 1657152000, 1657238400, 1657324800,
            1657411200, 1657497600, 1657584000, 1657670400, 1657756800,
            1657843200, 1657929600, 1658016000, 1658102400, 1658188800,
            1658275200, 1658361600, 1658448000, 1658534400, 1658620800,
            1658707200],
           dtype='int64', name='date')
[datetime.datetime(2022, 7, 5, 0, 0), datetime.datetime(2022, 7, 6, 0, 0), datetime.datetime(2022, 7, 7, 0, 0), datetime.datetime(2022, 7, 8, 0, 0), datetime.datetime(2022, 7, 9, 0, 0), datetime.datetime(2022, 7, 10, 0, 0), datetime.datetime(2022, 7, 11, 0, 0), datetime.datetime(2022, 7, 12, 

# Weather data


In [164]:
weather_july = pd.read_csv('../www/flaskapp/static/20220731_predictweather.csv')
weather_july

,id,latitude,longitude,temp0,wind_speed0,precipitation0,soil_moisture0,temp1,wind_speed1,precipitation1,...,precipitation28,soil_moisture28,temp29,wind_speed29,precipitation29,soil_moisture29,temp30,wind_speed30,precipitation30,soil_moisture30
0,0,49.006704,-122.727215,66.0,1.342596,0.0,21.38,68.0,1.248693,0.0,...,0.0,20.07,82.0,2.342153,0.0,19.86,81.0,2.209168,0.0,19.67
1,1,49.006704,-122.714526,62.0,1.940375,0.0,19.89,66.0,2.031342,0.0,...,0.0,17.03,76.0,1.605325,0.0,16.86,74.0,1.728974,0.0,16.72
2,2,49.006704,-122.676457,62.0,1.940375,0.0,19.89,66.0,2.031342,0.0,...,0.0,17.03,76.0,1.605325,0.0,16.86,74.0,1.728974,0.0,16.72
3,3,49.006704,-122.625699,62.0,1.940375,0.0,19.89,66.0,2.031342,0.0,...,0.0,17.03,76.0,1.605325,0.0,16.86,74.0,1.728974,0.0,16.72
4,4,49.006704,-122.613010,62.0,1.940375,0.0,19.89,66.0,2.031342,0.0,...,0.0,17.03,76.0,1.605325,0.0,16.86,74.0,1.728974,0.0,16.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129799,129799,45.555160,-122.245014,69.0,3.152320,0.0,21.58,69.0,2.931181,0.0,...,0.0,19.32,86.0,2.297099,0.0,19.20,84.0,2.863446,0.0,19.07
129800,129800,45.542471,-122.359220,69.0,3.152320,0.0,21.58,69.0,2.931181,0.0,...,0.0,19.32,86.0,2.297099,0.0,19.20,84.0,2.863446,0.0,19.07
129801,129801,45.542471,-122.333841,69.0,3.152320,0.0,21.58,69.0,2.931181,0.0,...,0.0,19.32,86.0,2.297099,0.0,19.20,84.0,2.863446,0.0,19.07
129802,129802,45.542471,-122.321151,69.0,3.152320,0.0,21.58,69.0,2.931181,0.0,...,0.0,19.32,86.0,2.297099,0.0,19.20,84.0,2.863446,0.0,19.07


In [165]:
cur_date = datetime.date(2022, 7, 1)
delta = datetime.timedelta(days=1)

weather_july_list=  []

for i in range(30):
    cur_day = weather_july[[f'temp{i}', f'wind_speed{i}', f'precipitation{i}', f'soil_moisture{i}', 'latitude', 'longitude']]
    cur_date_col = pd.DataFrame([cur_date]*len(cur_day), columns=['date'])
    cur_day = cur_day.join(cur_date_col)
    cur_date += delta
    cur_day.columns = pd.Series(['temp', 'wind_speed', 'precipitation', 'soil_moisture', 'latitude', 'longitude', 'date'])
    weather_july_list.append(cur_day)
    


In [166]:
all_weather = pd.concat(weather_july_list)
all_weather['date'] = all_weather.apply(lambda r: int(time.mktime(r['date'].timetuple())), axis=1)

In [167]:
all_weather.set_index(['date', 'latitude', 'longitude'], inplace=True)
all_weather

temp  wind_speed  precipitation  \
date       latitude  longitude                                      
1656633600 49.006704 -122.727215  66.0    1.342596            0.0   
                     -122.714526  62.0    1.940375            0.0   
                     -122.676457  62.0    1.940375            0.0   
                     -122.625699  62.0    1.940375            0.0   
                     -122.613010  62.0    1.940375            0.0   
...                                ...         ...            ...   
1659139200 45.555160 -122.245014  86.0    2.297099            0.0   
           45.542471 -122.359220  86.0    2.297099            0.0   
                     -122.333841  86.0    2.297099            0.0   
                     -122.321151  86.0    2.297099            0.0   
                     -122.308462  86.0    2.297099            0.0   

                                  soil_moisture  
date       latitude  longitude                   
1656633600 49.006704 -122.727215          21.38  
                     -122.714526          19.89  
                     -122.676457          19.89  
                     -122.625699          19.89  
                     -122.613010          19.89  
...                                         ...  
1659139200 45.555160 -122.245014          19.20  
           45.542471 -122.359220          19.20  
                     -122.333841          19.20  
                     -122.321151          19.20  
                     -122.308462          19.20  

[3894120 rows x 4 columns]

# Data assimilation


In [168]:
all_predictions

dayConfidence
date       latitude  longitude                 
1658793600 47.699686 -123.171347       0.728846
           47.686996 -123.234795       0.728846
                     -123.222105       0.728846
                     -123.184037       0.728846
                     -123.171347       0.728846
...                                         ...
1659139200 46.951005 -122.206946       0.477894
                     -122.194256       0.477894
                     -122.181567       0.477894
                     -122.168877       0.477894
                     -122.156188       0.477894

[5000 rows x 1 columns]

In [169]:
all_historical

dayConfidence
date       latitude  longitude                
1656979200 48.575261 -118.108238            -1
           47.801202 -117.105767            -1
           46.176946 -119.872078            -1
           48.194576 -120.506553            -1
           48.283403 -118.336649            -1
...                                        ...
1658707200 48.321471 -117.879827            -1
           47.648928 -117.448384            -1
           47.801202 -121.293302            -1
           48.042302 -118.894987            -1
           47.179416 -123.145968            -1

[161 rows x 1 columns]

In [170]:
all_weather

temp  wind_speed  precipitation  \
date       latitude  longitude                                      
1656633600 49.006704 -122.727215  66.0    1.342596            0.0   
                     -122.714526  62.0    1.940375            0.0   
                     -122.676457  62.0    1.940375            0.0   
                     -122.625699  62.0    1.940375            0.0   
                     -122.613010  62.0    1.940375            0.0   
...                                ...         ...            ...   
1659139200 45.555160 -122.245014  86.0    2.297099            0.0   
           45.542471 -122.359220  86.0    2.297099            0.0   
                     -122.333841  86.0    2.297099            0.0   
                     -122.321151  86.0    2.297099            0.0   
                     -122.308462  86.0    2.297099            0.0   

                                  soil_moisture  
date       latitude  longitude                   
1656633600 49.006704 -122.727215          21.38  
                     -122.714526          19.89  
                     -122.676457          19.89  
                     -122.625699          19.89  
                     -122.613010          19.89  
...                                         ...  
1659139200 45.555160 -122.245014          19.20  
           45.542471 -122.359220          19.20  
                     -122.333841          19.20  
                     -122.321151          19.20  
                     -122.308462          19.20  

[3894120 rows x 4 columns]

In [171]:
all_historical =  all_historical.join(all_weather)
all_historical

dayConfidence  temp  wind_speed  \
date       latitude  longitude                                     
1656979200 45.682055 -122.295772            -1  62.0    1.226671   
           45.999293 -116.966183            -1  66.0    1.457985   
           46.176946 -119.872078            -1  74.0    1.332870   
           46.316530 -117.968653            -1  64.0    1.615105   
           46.900247 -123.120589            -1  70.0    1.706152   
...                                        ...   ...         ...   
1658707200 47.483964 -121.039512            -1  75.0    2.920214   
           47.648928 -117.448384            -1  90.0    1.257605   
           47.801202 -121.293302            -1  77.0    2.425577   
           48.042302 -118.894987            -1  84.0    2.707216   
           48.321471 -117.879827            -1  79.0    2.965815   

                                  precipitation  soil_moisture  
date       latitude  longitude                                  
1656979200 45.682055 -122.295772         0.4496          21.17  
           45.999293 -116.966183         0.0736          30.21  
           46.176946 -119.872078         0.0000          13.55  
           46.316530 -117.968653         0.0968          30.50  
           46.900247 -123.120589         0.4304          20.23  
...                                         ...            ...  
1658707200 47.483964 -121.039512         0.0000          19.71  
           47.648928 -117.448384         0.0000          20.05  
           47.801202 -121.293302         0.0000          19.66  
           48.042302 -118.894987         0.0000          23.75  
           48.321471 -117.879827         0.0000          24.98  

[161 rows x 5 columns]

In [172]:
assembled_predictions = all_predictions.join(all_weather)
assembled_predictions

dayConfidence  temp  wind_speed  \
date       latitude  longitude                                      
1658793600 47.699686 -123.171347       0.728846  76.0    1.422571   
           47.686996 -123.234795       0.728846  84.0    1.865123   
                     -123.222105       0.728846  84.0    1.865123   
                     -123.184037       0.728846  76.0    1.422571   
                     -123.171347       0.728846  76.0    1.422571   
...                                         ...   ...         ...   
1659139200 46.951005 -122.206946       0.477894  82.0    2.719667   
                     -122.194256       0.477894  82.0    2.719667   
                     -122.181567       0.477894  82.0    2.719667   
                     -122.168877       0.477894  82.0    2.719667   
                     -122.156188       0.477894  82.0    2.719667   

                                  precipitation  soil_moisture  
date       latitude  longitude                                  
1658793600 47.699686 -123.171347            0.0          28.26  
           47.686996 -123.234795            0.0          20.71  
                     -123.222105            0.0          20.71  
                     -123.184037            0.0          28.26  
                     -123.171347            0.0          28.26  
...                                         ...            ...  
1659139200 46.951005 -122.206946            0.0          15.42  
                     -122.194256            0.0          15.42  
                     -122.181567            0.0          15.42  
                     -122.168877            0.0          15.42  
                     -122.156188            0.0          15.42  

[5000 rows x 5 columns]

In [173]:
assembled_data = pd.concat([all_historical, assembled_predictions])
assembled_data

dayConfidence  temp  wind_speed  \
date       latitude  longitude                                     
1656979200 45.682055 -122.295772            -1  62.0    1.226671   
           45.999293 -116.966183            -1  66.0    1.457985   
           46.176946 -119.872078            -1  74.0    1.332870   
           46.316530 -117.968653            -1  64.0    1.615105   
           46.900247 -123.120589            -1  70.0    1.706152   
...                                        ...   ...         ...   
1659139200 46.951005 -122.206946      0.477894  82.0    2.719667   
                     -122.194256      0.477894  82.0    2.719667   
                     -122.181567      0.477894  82.0    2.719667   
                     -122.168877      0.477894  82.0    2.719667   
                     -122.156188      0.477894  82.0    2.719667   

                                  precipitation  soil_moisture  
date       latitude  longitude                                  
1656979200 45.682055 -122.295772         0.4496          21.17  
           45.999293 -116.966183         0.0736          30.21  
           46.176946 -119.872078         0.0000          13.55  
           46.316530 -117.968653         0.0968          30.50  
           46.900247 -123.120589         0.4304          20.23  
...                                         ...            ...  
1659139200 46.951005 -122.206946         0.0000          15.42  
                     -122.194256         0.0000          15.42  
                     -122.181567         0.0000          15.42  
                     -122.168877         0.0000          15.42  
                     -122.156188         0.0000          15.42  

[5161 rows x 5 columns]

In [174]:
for q in [datetime.datetime.fromtimestamp(i) for i in assembled_data.index.get_level_values(0).unique()]:
    print(q)

2022-07-05 00:00:00
2022-07-06 00:00:00
2022-07-07 00:00:00
2022-07-08 00:00:00
2022-07-09 00:00:00
2022-07-10 00:00:00
2022-07-11 00:00:00
2022-07-12 00:00:00
2022-07-13 00:00:00
2022-07-14 00:00:00
2022-07-15 00:00:00
2022-07-16 00:00:00
2022-07-17 00:00:00
2022-07-18 00:00:00
2022-07-19 00:00:00
2022-07-20 00:00:00
2022-07-21 00:00:00
2022-07-22 00:00:00
2022-07-23 00:00:00
2022-07-24 00:00:00
2022-07-25 00:00:00
2022-07-26 00:00:00
2022-07-27 00:00:00
2022-07-28 00:00:00
2022-07-29 00:00:00
2022-07-30 00:00:00


In [175]:
assembled_data = assembled_data.reset_index(level=2)
assembled_data = assembled_data.reset_index(level=1)
assembled_data = assembled_data.reset_index(level=0)
assembled_data['SNo'] = np.arange(len(assembled_data))
assembled_data['date'] = assembled_data['date']*1000
assembled_data

,date,latitude,longitude,dayConfidence,temp,wind_speed,precipitation,soil_moisture,SNo
0,1656979200000,45.682055,-122.295772,-1,62.0,1.226671,0.4496,21.17,0
1,1656979200000,45.999293,-116.966183,-1,66.0,1.457985,0.0736,30.21,1
2,1656979200000,46.176946,-119.872078,-1,74.0,1.332870,0.0000,13.55,2
3,1656979200000,46.316530,-117.968653,-1,64.0,1.615105,0.0968,30.50,3
4,1656979200000,46.900247,-123.120589,-1,70.0,1.706152,0.4304,20.23,4
...,...,...,...,...,...,...,...,...,...
5156,1659139200000,46.951005,-122.206946,0.477894,82.0,2.719667,0.0000,15.42,5156
5157,1659139200000,46.951005,-122.194256,0.477894,82.0,2.719667,0.0000,15.42,5157
5158,1659139200000,46.951005,-122.181567,0.477894,82.0,2.719667,0.0000,15.42,5158
5159,1659139200000,46.951005,-122.168877,0.477894,82.0,2.719667,0.0000,15.42,5159


In [176]:
features = assembled_data.apply(
    lambda row: Feature(properties={"id": row["SNo"], "mag": row["dayConfidence"], "time": row["date"], 'confidence': row['dayConfidence'], 'temp': row['temp'], 'wind_speed': row['wind_speed'], 'precipitation': row['precipitation'], 'soil_moisture': row['soil_moisture']}, geometry=Point((float(row["longitude"]), float(row["latitude"])))), axis=1
).tolist()

In [177]:
feature_collection = FeatureCollection(features=features, properties={"name": "assembled_data"})

In [178]:
with open(f'/home/tejabalu/www/flaskapp/static/assembled_data{nlargest}largest.geojson', 'w', encoding='utf-8') as f:
    f.write(json.dumps(feature_collection, ensure_ascii=False))
